## Title & Overview

- Project title: Satellite Metadata Search Agent (STAC Agent)
- Problem statement:
  - Provide natural-language search over satellite imagery metadata (Sentinel-2 STAC) using Google ADK.
  - Users can specify area, time range, and cloud-cover constraints in free-form text.
- High-level summary:
  - Resolve AOI (Area of Interest) from place names using a predefined AOI catalog.
  - Interpret time range and cloud-cover constraints from text.
  - Call a STAC search tool against Sentinel-2 metadata.
  - Return results in a human-readable table.


## Background & Objectives

- Competition: Agents Intensive Capstone Project (Kaggle).
- Goal of the capstone:
  - Build an agent that solves a real-world problem or improves everyday productivity.
- This agent:
  - Focuses on Earth observation (EO) data accessibility.
  - Targets analysts and developers who want to quickly explore Sentinel-2 imagery without writing STAC queries.
- Objectives of this Notebook:
  - Describe the system design and agent architecture.
  - Demonstrate core behaviors (normal search and clarification).
  - Summarize evaluation and limitations.


## System Design & Architecture
- Modules: agent/stac_agent_adk.py (root agent), agent/prompts.py (instructions), aoi/aoi_catalog.py (AOI resolver), tools/stac_search.py (STAC API), scripts/run_eval.py (eval runner).
- Flow: user → resolve AOI → build search params → call Element84 Earth Search v1 → format table.

![Agent flow](fig/flow.png)

## Environment & Dependencies (Kaggle)

This Notebook is intended to run directly on Kaggle for the Agents Intensive Capstone Project.

### Kaggle Runtime
- Python 3.x (default Kaggle environment)
- Internet access is required for live Gemini (Google ADK) and STAC API calls.
  - If Internet is disabled, live demos will be skipped.

### API Keys
- Set `GOOGLE_API_KEY` in Kaggle:
  - Add-ons → Secrets → New Secret  
  - Key: `GOOGLE_API_KEY`
- Demo cells check this variable and will skip live agent calls if it is missing.

```python
import os
if not os.getenv("GOOGLE_API_KEY"):
    print("GOOGLE_API_KEY not set — skipping live agent calls.")


In [ ]:
# Code cell: install dependencies (adjust versions as needed)
! pip install google-adk python-dotenv pandas shapely requests tabulate



INFO: pip is looking at multiple versions of google-auth to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement cachetools<6.0,>=2.0.0 (from google-auth) (from versions: none)
ERROR: No matching distribution found for cachetools<6.0,>=2.0.0


In [7]:
# Code cell: unpack submission bundle from Kaggle Dataset
!ls /kaggle/input  # TODO: find dataset name
!ls /kaggle/input/google5days/capstone  # TODO: find dataset name
# !tar -xzf /kaggle/input/google5days/capstone/submission.tar.gz -C /kaggle/working/
! cp -pr /kaggle/input/google5days/capstone /kaggle/working
!ls /kaggle/working


google5days
aoi  doc  pyproject.toml  README.md  sample_stac_results  scripts  src
capstone


In [8]:
# Code cell: add project to Python path
import sys, pathlib
PROJECT_ROOT = pathlib.Path("/kaggle/working/capstone")
sys.path.append(str(PROJECT_ROOT / "src"))


## Configuration & API Keys
- Gemini API key stored as Kaggle Secret GOOGLE_API_KEY.
- If key missing, skip agent demos to avoid failures.

In [12]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GOOGLE_API_KEY")


## Agent Components

The agent is defined in `capstone.agent.stac_agent_adk.create_agent()`.

### Root Agent
- Built with Google ADK.
- Handles natural-language queries, clarification, tool selection, and final formatting.

### Tools
- **resolve_aoi**  
  Converts location text (e.g., "eastern Hokkaido") into an AOI ID and bounding box using `aoi_catalog.json`.

- **search_satellite_scenes**  
  Queries Sentinel-2 L2A metadata through a STAC-compatible API  
  (collection: `sentinel-2-l2a`).

### Model
- Default model is set via environment variable `MODEL_NAME` (e.g., `"gemini-2.0-flash"`).
- Can be overridden:

```python
import os
os.environ["MODEL_NAME"] = "gemini-3b"  # optional override
```

## Component Flow (Text Diagram)

```scss
User Query
   ↓
Root Agent (ADK)
   ↓
[ resolve_aoi | parse dates | parse cloud cover ]
   ↓
search_satellite_scenes (STAC API)
   ↓
Tabular Formatting
   ↓
Final Answer
```


## Demo: Normal Query Behavior

In [ ]:
# Code cell: run a normal query through the agent
import asyncio
from google.genai import types
from capstone.agent.stac_agent_adk import create_runner_async

async def run_query(query: str, user_id: str = "kaggle_user", session_id: str = "demo_session"):
    runner, _ = await create_runner_async(user_id=user_id, session_id=session_id)
    user_msg = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=user_msg)
    final_text = None
    for ev in events:
        if ev.is_final_response() and ev.content and ev.content.parts:
            final_text = ev.content.parts[0].text
    return final_text or "TODO: inspect intermediate events for details."

if os.getenv("GOOGLE_API_KEY"):
    normal_query = "TODO: fill normal query text"
    print(asyncio.run(run_query(normal_query)))
else:
    print("Skipping agent call because GOOGLE_API_KEY is missing.")


- TODO: Add markdown commentary on expected table columns (id, datetime UTC, cloud_cover, preview_url).
## Demo: Clarification Behavior
- Ambiguous query placeholder: “Show me cloud-free images around Sapporo in September.”
- Expected: agent asks for explicit date range before search.

In [ ]:
# Code cell: trigger clarification behavior
if os.getenv("GOOGLE_API_KEY"):
    unclear_query = "TODO: fill ambiguous query text"
    print(asyncio.run(run_query(unclear_query, session_id="clarify_session")))
else:
    print("Skipping agent call because GOOGLE_API_KEY is missing.")


- TODO: Describe expected back-and-forth (resolve_aoi call; request for precise dates/clouds).
## Evaluation Notes
- Local tests: .venv/bin/pytest tests/unit -q (AOI catalog, wiring).
- ADK smoke: .venv/bin/python -m capstone.scripts.run_eval (loads normal/clarification/boundary evalsets).
- Full ADK CLI: adk eval src/capstone/scripts/eval ... --print_detailed_results > result.txt.
- TODO: Paste your actual results/metrics; note any failures or skipped cases.
- TODO: List known limitations / future fixes.
## Conclusion & Future Work
- TODO: Summarize agent value (NL → STAC search, deterministic tool use).
- TODO: Add roadmap items (more AOIs, additional sensors, caching, richer previews).
